This tests the accuracy of a differential operator on a polynomial of the given order. If the operator is working to said order, then the `Actual` and `Theoretical` outputs should match (except possibly at the edges.)

In [1]:
import os.path
from scipy import *
import numpy as np
from numpy import *
from numpy import linalg as LA
from scipy import linalg as LA2
import sympy as sympy
import sys as sys
import time
import matplotlib.pyplot as plt
import itertools as it
from IPython.core.display import HTML

sys.path.append('/Users/sashacurcic/SashasDirectory/ANAG/FV_MG/')
from Modules import BasicTools as BT
from Modules import WaveTools as WT
from Modules import PlotTools as PT
from Modules import FFTTools as FFTT
from Modules import OperatorTools as OT
from Modules import GridTransferTools as GTT
from Modules import TestTools as TT
from Modules import SolverTools as ST

display(HTML("<style>pre { white-space: pre !important; }</style>"))
np.set_printoptions( linewidth = 10000, threshold = 100000)

This cell takes all of the input parameters.

In [2]:
# Grid parameters.
nh_min = 16
refRatio = 2
cfInter = int(nh_min / refRatio)

L = 1
locs = [0.5]
epsilons = [1, 0.5] # [1, 1]
mus = [1, 0.5]

# Time integration parameters.
deriv = 'UD'
# RK = 4
orders = 5

This cell contains the relevant switch information.

In [3]:
# TimeIntFunc, DiffFunc = TT.SolverSwitch(deriv, RK)

This cell instantiates the Grid and the Physics objects.

In [4]:
omega = BT.Grid(nh_min)
finepart = list(np.arange(int(nh_min / 4), int(3 * nh_min / 4)))
omega.AddPatch(refRatio, finepart)
degFreed = omega.degFreed


physics = BT.PhysProps(omega, epsilons, mus, locs, L)

This cell runs a test on the derivative operator on a cell-averaged polynomial which matches the given order.  
$P (x) = \sum_{k = 0}^{o} x^{k}$  
$<P (x)> = \frac{1}{\Delta x} \int_{x}^{x + \Delta x} P (x')$d$x'$  
$<P (x)> = \frac{1}{\Delta x} \sum_{k = 1}^{o} \frac{1}{k} [(x + \Delta x)^{k} - x^{k}]$  
$<\frac{d P (x)}{d x}> = \sum_{k = 1}^{o} [(x + \Delta x)^{k} - x^{k}]$  

In [5]:
for o in range(orders):
    order = 2*o + 1
    print('Order:', order)
    TT.DerivPolyTest(omega, deriv, order, deriv = 0, printOut = False, iterate = True)

Order: 1
PASS
[16.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]

Order: 3
PASS
[38.73958333 -7.59375    -0.          0.          0.          0.         -0.          0.          0.         -0.         -0.          0.         -0.         -0.          0.         -0.         -0.          0.         -0.          0.          0.         -0.         -0.         16.85416667]

Order: 5
PASS
[ 59.71186388 -16.29315728   2.44608019   0.           0.          -0.          -0.           0.          -0.           0.           0.          -0.           0.          -0.          -0.           0.          -0.          -0.           0.          -0.           0.          -0.          -4.37048747  38.50570068]

Order: 7
FAIL
[ 79.80822798 -25.08085821   6.0773869   -0.71129335  -0.0023316    0.          -0.           0.          -0.          -0.           0.          -0.          -0.           0.           0.          -0.          -0.          -0.           0.

In [6]:
coefs = 1. / (np.arange(order + 1) + 1)
coefs = np.append(1, coefs)[::-1]
P = np.poly1d(coefs) - 1
print(coefs)
print(P)
print('')

[0.1        0.11111111 0.125      0.14285714 0.16666667 0.2        0.25       0.33333333 0.5        1.         1.        ]
     10          9         8          7          6       5        4
0.1 x  + 0.1111 x + 0.125 x + 0.1429 x + 0.1667 x + 0.2 x + 0.25 x
           3       2
 + 0.3333 x + 0.5 x + 1 x



In [7]:
def FacePolyTest(omega, physics, diff, orderIn, Ng, coefs = [], printOut = True):
    order = orderIn
    errorLoc = 'ERROR:\nTestTools:\nFacePolyTest:\n'
    degFreed = omega.degFreed
    hs = omega.h
    
    halfDeg = int(degFreed / 2)
    
    if (coefs == []):
        coefs = 1. / (np.arange(order + 1) + 1)
        coefs = np.append(1, coefs)[::-1]
    else:
        errorMess = BT.CheckSize(order, coefs, nName = 'order', matricaName = 'coefs')
        if (errorMess != ''):
            sys.exit(errorLoc + errorMess)
    nh_max = omega.nh_max
    waves = WT.MakeWaves(omega)
    
    x = omega.xNode
#     x = np.append(-hs[0], x)
#     hs = np.append(hs[0], hs)
    P = np.poly1d(coefs) - 1
    waveform = (P(x[1:]) - P(x[:-1])) / hs
    if (Ng != 0):
        leftPiece = np.zeros(Ng, float)
        rightPiece = np.full(Ng, waveform[-1])
        
        wG1, wG2 = OT.GhostCellsJump(omega, physics, waveform, Ng, order)
        waveform1 = waveform[:halfDeg]
        waveform2 = waveform[halfDeg:]
        waveform1 = np.hstack([leftPiece, waveform1, wG1])
        waveform2 = np.hstack([wG2, waveform2, rightPiece])
        waveform = np.hstack([leftPiece, waveform, rightPiece])
    
    p = np.poly1d(np.ones(order + 1)) - 1
    waveformDeriv = (p(x[1:]) - p(x[:-1])) / hs
    
    const = -np.eye(degFreed)
    faceOp1, faceOp2 = OT.FaceOp(omega, order, diff, 'R', Ng)# DiffFunc(omega, 0, waveform, const, order)
    hMat = OT.StepMatrix(omega)
    hMat1 = hMat[:halfDeg, :halfDeg]
    hMat2 = hMat[halfDeg:, halfDeg:]
    np.set_printoptions(suppress = True)
    waveFace1 = faceOp1 @ waveform1
    waveFace2 = faceOp2 @ waveform2
    wavederiv1 = hMat1 @ (waveFace1[1:] - waveFace1[:-1])
    wavederiv2 = hMat2 @ (waveFace2[1:] - waveFace2[:-1])
    
#     print(derivOp)
    error1 = np.round(waveformDeriv[:halfDeg] - wavederiv1, 11)
    error2 = np.round(waveformDeriv[halfDeg:] - wavederiv2, 11)

    print('')
    if (printOut):
        print('x:')
        print(x)
        print('')
        print('Polynomial Function:')
        print('<p(x)> =\n', P)
        print('<p(x)> =\n', waveform)
        print('')
        print('Polynomial Derivative:')
        print('<dp(x)/dx> =\n', p)
        print('Theoretical:')
        print('<dp(x)/dx> =\n', waveformDeriv)
        print('Actual:')
        print('<dp(x)/dx> =\n', wavederiv1)
        print(wavederiv2)
    print('Difference Between Actual and Theoretical:')
    print(error1)
    print(error2)
    print('')
    print('')
    return

In [9]:
order = 3
if (order == 1):
    Ng = 0
else:
    Ng = order - 2
FacePolyTest(omega, physics, deriv, order, Ng, printOut = True)



x:
[0.      0.0625  0.125   0.1875  0.25    0.28125 0.3125  0.34375 0.375   0.40625 0.4375  0.46875 0.5     0.53125 0.5625  0.59375 0.625   0.65625 0.6875  0.71875 0.75    0.8125  0.875   0.9375  1.     ]

Polynomial Function:
<p(x)> =
       4          3       2
0.25 x + 0.3333 x + 0.5 x + 1 x
<p(x)> =
 [0.                 1.0326131184895833 1.1037801106770833 1.1849568684895835 1.277608235677083  1.3550694783528652 1.411328633626301  1.4712804158528652 1.5351079305013027 1.6029942830403634 1.675122578938801  1.751675923665367  1.832837422688801  1.9187901814778634 2.0097173055013045 2.1058019002278634 2.207227071126301  2.314175923665367  2.4268315633137973 2.545377095540367  2.6699956258138045 2.869527180989582  3.157491048177082  3.4730428059895857 3.8176472981770786 3.8176472981770786]

Polynomial Derivative:
<dp(x)/dx> =
    3     2
1 x + 1 x + 1 x
Theoretical:
<dp(x)/dx> =
 [1.06640625   1.21484375   1.38671875   1.58203125   1.7431640625 1.8583984375 1.9794921875 2.1064453125